In [1]:
import pandas as pd
import json
import pickle

## Create profiles table

In [2]:
with open('profiles.json', 'r') as f:
    data = json.load(f)

In [3]:
data[0]

{'player_id': 1809,
 'name': 'Robert Blackmon',
 'position': 'DB',
 'height': '6-0',
 'weight': '208',
 'current_team': None,
 'birth_date': '1967-05-12',
 'birth_place': 'Bay City, TX',
 'death_date': None,
 'college': 'Baylor',
 'high_school': 'Van Vleck, TX',
 'draft_team': 'Seattle Seahawks',
 'draft_round': '2',
 'draft_position': '34',
 'draft_year': '1990',
 'current_salary': None,
 'hof_induction_year': None}

In [4]:
cleaned_data = []
for entry in data:
    cleaned_data.append({'playerID': entry['player_id'], 'name': entry['name'].strip(' '), 'position': entry['position'].split('-')[0],
                        'team': entry['current_team']})

In [5]:
profile = pd.DataFrame(cleaned_data)
profile.set_index('playerID', inplace=True)

In [6]:
profile.head()

,name,position,team
playerID,,,
1809,Robert Blackmon,DB,None
23586,Dean Wells,LB,None
355,Kiko Alonso,ILB,Miami Dolphins
18182,Steve Ramsey,QB,None
16250,Cory Nelms,CB,None


## Read in Games Data

In [7]:
with open('games.json', 'r') as f:
    games = json.load(f)

In [8]:
gameframe = pd.DataFrame(games)
gameframe = gameframe[gameframe['year'] == '2016']  # Limit to 2016 year

In [9]:
gameframe.head()

,player_id,year,date,game_number,age,team,game_location,opponent,game_won,player_team_score,...,defense_interception_yards,defense_interception_touchdowns,defense_safeties,point_after_attemps,point_after_makes,field_goal_attempts,field_goal_makes,punting_attempts,punting_yards,punting_blocked
290,355,2016,2016-09-11,1,26-028,MIA,A,SEA,False,10,...,0,0,0,0,0,0,0,0,0,0
291,355,2016,2016-09-18,2,26-035,MIA,A,NWE,False,24,...,0,0,0,0,0,0,0,0,0,0
292,355,2016,2016-09-25,3,26-042,MIA,H,CLE,True,30,...,0,0,0,0,0,0,0,0,0,0
293,355,2016,2016-09-29,4,26-046,MIA,A,CIN,False,7,...,0,0,0,0,0,0,0,0,0,0
294,355,2016,2016-10-09,5,26-056,MIA,H,TEN,False,17,...,0,0,0,0,0,0,0,0,0,0


## Create Offense Table

In [10]:
offense = gameframe[['player_id', 'game_number', 'passing_yards', 'rushing_yards', 'receiving_yards', 
                          'passing_touchdowns', 'receiving_touchdowns', 'rushing_touchdowns', 'passing_interceptions']]

offense = offense.rename(columns={'player_id': 'playerID', 'game_number': 'gameNumber', 'passing_yards': 'passYards', 
                                  'rushing_yards': 'rushYards', 'receiving_yards': 'recYards', 
                                  'passing_interceptions': 'interceptions'})

In [11]:
offense['touchdowns'] = offense['passing_touchdowns'] + offense['receiving_touchdowns'] + offense['rushing_touchdowns']
offense.drop(['passing_touchdowns', 'receiving_touchdowns', 'rushing_touchdowns'], axis=1, inplace=True)

In [12]:
offense.head()

,playerID,gameNumber,passYards,rushYards,recYards,interceptions,touchdowns
290,355,1,0,0,0,0,0
291,355,2,0,0,0,0,0
292,355,3,0,0,0,0,0
293,355,4,0,0,0,0,0
294,355,5,0,0,0,0,0


In [13]:
offense['gameNumber'] = offense['gameNumber'].astype(int)

In [14]:
offense.head()

,playerID,gameNumber,passYards,rushYards,recYards,interceptions,touchdowns
290,355,1,0,0,0,0,0
291,355,2,0,0,0,0,0
292,355,3,0,0,0,0,0
293,355,4,0,0,0,0,0
294,355,5,0,0,0,0,0


In [15]:
offense = offense[offense['gameNumber'] < 17]

In [16]:
offensive_profiles = profile[(profile['position'] == 'QB') | (profile['position'] == 'WR') | (profile['position'] == 'RB') | (profile['position'] == 'TE')]

In [17]:
merged = pd.merge(offense, offensive_profiles, how='inner', on='playerID')

In [18]:
merged.head()

,playerID,gameNumber,passYards,rushYards,recYards,interceptions,touchdowns,name,position,team
0,18011,4,0,0,7,0,0,MyCole Pruitt,TE,None
1,18011,5,0,0,0,0,0,MyCole Pruitt,TE,None
2,18011,15,0,0,0,0,0,MyCole Pruitt,TE,None
3,18011,16,0,0,6,0,0,MyCole Pruitt,TE,None
4,19449,1,0,0,49,0,0,Emmanuel Sanders,WR,Denver Broncos


In [19]:
merged.dropna(axis=0, inplace=True)

In [20]:
merged.head()

,playerID,gameNumber,passYards,rushYards,recYards,interceptions,touchdowns,name,position,team
4,19449,1,0,0,49,0,0,Emmanuel Sanders,WR,Denver Broncos
5,19449,2,0,0,39,0,0,Emmanuel Sanders,WR,Denver Broncos
6,19449,3,0,0,117,0,2,Emmanuel Sanders,WR,Denver Broncos
7,19449,4,0,0,88,0,1,Emmanuel Sanders,WR,Denver Broncos
8,19449,5,0,0,80,0,0,Emmanuel Sanders,WR,Denver Broncos


In [21]:
merged.drop(['playerID', 'team'], axis=1, inplace=True)

merged = merged.rename(columns={'name': 'playerID'})

In [22]:
merged.head()

,gameNumber,passYards,rushYards,recYards,interceptions,touchdowns,playerID,position
4,1,0,0,49,0,0,Emmanuel Sanders,WR
5,2,0,0,39,0,0,Emmanuel Sanders,WR
6,3,0,0,117,0,2,Emmanuel Sanders,WR
7,4,0,0,88,0,1,Emmanuel Sanders,WR
8,5,0,0,80,0,0,Emmanuel Sanders,WR


In [23]:
with open('offense.pickle', 'wb') as f:
    pickle.dump(merged, f)

## Create Defense Table

In [24]:
defense = gameframe[['player_id', 'game_number', 'defense_sacks', 'defense_interceptions', 'defense_interception_touchdowns',
                    'defense_safeties', 'punting_blocked']]

defense = defense.rename(columns={'player_id': 'playerID', 'game_number': 'gameNumber', 'defense_interceptions': 'interceptions',
                                 'defense_interception_touchdowns': 'touchdowns', 'defense_safeties': 'safeties',
                                 'punting_blocked': 'blocks', 'defense_sacks': 'sacks'})

In [25]:
defense.head()

,playerID,gameNumber,sacks,interceptions,touchdowns,safeties,blocks
290,355,1,0.0,0,0,0,0
291,355,2,0.0,0,0,0,0
292,355,3,0.0,0,0,0,0
293,355,4,0.0,0,0,0,0
294,355,5,0.0,0,0,0,0


In [26]:
defense['gameNumber'] = defense['gameNumber'].astype(int)

In [27]:
defense = defense[defense['gameNumber'] < 17]

In [28]:
defense_profiles = profile[(profile['position'] != 'QB') & (profile['position'] != 'WR') & (profile['position'] != 'RB') & (profile['position'] != 'TE')]

In [29]:
defense = pd.merge(defense, defense_profiles, how='inner', on='playerID')

In [30]:
defense.head()

,playerID,gameNumber,sacks,interceptions,touchdowns,safeties,blocks,name,position,team
0,355,1,0.0,0,0,0,0,Kiko Alonso,ILB,Miami Dolphins
1,355,2,0.0,0,0,0,0,Kiko Alonso,ILB,Miami Dolphins
2,355,3,0.0,0,0,0,0,Kiko Alonso,ILB,Miami Dolphins
3,355,4,0.0,0,0,0,0,Kiko Alonso,ILB,Miami Dolphins
4,355,5,0.0,0,0,0,0,Kiko Alonso,ILB,Miami Dolphins


In [31]:
defense.dropna(axis=0, inplace=True)

In [32]:
defense.head()

,playerID,gameNumber,sacks,interceptions,touchdowns,safeties,blocks,name,position,team
0,355,1,0.0,0,0,0,0,Kiko Alonso,ILB,Miami Dolphins
1,355,2,0.0,0,0,0,0,Kiko Alonso,ILB,Miami Dolphins
2,355,3,0.0,0,0,0,0,Kiko Alonso,ILB,Miami Dolphins
3,355,4,0.0,0,0,0,0,Kiko Alonso,ILB,Miami Dolphins
4,355,5,0.0,0,0,0,0,Kiko Alonso,ILB,Miami Dolphins


In [33]:
defense = defense.groupby(['team', 'gameNumber'], as_index=False)[['sacks', 'interceptions', 'touchdowns', 'safeties', 'blocks']].sum()

In [34]:
defense.head()

,team,gameNumber,sacks,interceptions,touchdowns,safeties,blocks
0,Arizona Cardinals,1,2.0,0,0,0,0
1,Arizona Cardinals,2,2.0,2,0,0,0
2,Arizona Cardinals,3,2.0,1,0,1,0
3,Arizona Cardinals,4,2.0,0,0,0,0
4,Arizona Cardinals,5,2.0,0,0,0,0


In [35]:
defense = defense.rename(columns={'team': 'playerID'})

In [36]:
defense.head()

,playerID,gameNumber,sacks,interceptions,touchdowns,safeties,blocks
0,Arizona Cardinals,1,2.0,0,0,0,0
1,Arizona Cardinals,2,2.0,2,0,0,0
2,Arizona Cardinals,3,2.0,1,0,1,0
3,Arizona Cardinals,4,2.0,0,0,0,0
4,Arizona Cardinals,5,2.0,0,0,0,0


In [37]:
with open('defense.pickle', 'wb') as f:
    pickle.dump(defense, f)

In [38]:
defense.to_csv('defense.csv')

In [41]:
merged.to_csv('offense.csv')